## E-model optimisation with BluePyEModel

This notebook will show how to use BluePyEModel to optimize an e-model using the pipeline.

For more detailed used-cases of BluePyOpt, refer the examples present in the examples directory of BluePyOpt at https://github.com/BlueBrain/BluePyOpt.

Author of this script: Tanguy Damart @ Blue Brain Project

In [ ]:
import logging

logger = logging.getLogger()
logging.basicConfig(
    level=logging.INFO,
    handlers=[logging.StreamHandler()]
)

In [ ]:
import shutil
import os

if os.path.exists('./config'):
    shutil.rmtree('./config')
    
if os.path.exists('./mechanisms'):
    shutil.rmtree('./mechanisms')
    
shutil.copytree('../../tests/config', './config')
shutil.copytree('../../tests/mechanisms', './mechanisms')

## Instantiate the Pipeline:

The pipeline has three ways of accessing the data required for optimisation: either using an PostrgreSQL database ("sql"), using Nexus ("nexus"), or using the old style config dictionary that relies on recipes ("singlecell"). In this example, we will use this last way, that refer to as "singlecell".

In [ ]:
db_api = 'singlecell'

The name of the e-model and species have to be specified. The name of the e-model has to match exactly the entry of the recipes.json when using the "singlecell" data access. With "singlecell", specifying the species of the model is required but inconsequential.

In [ ]:
emodel = 'cADpyr_L5TPC'
species = 'rat'

To set where the pipeline should save the results of the optimization as well as to know where are the mechanisms (.mod files) and where to store the final models, some setup is needed:

In [ ]:
import pathlib

working_dir = pathlib.Path('./')
mechanisms_dir = working_dir / 'mechanisms'
recipes_path = 'config/recipes.json'
final_path = './final.json'

Finally, with all these information, let's instantiate the pipeline:

In [ ]:
from bluepyemodel.emodel_pipeline.emodel_pipeline import EModel_pipeline

pipeline = EModel_pipeline(
    emodel=emodel,
    species=species,
    db_api=db_api,
    mechanisms_dir=mechanisms_dir,
    working_dir=working_dir,
    recipes_path=recipes_path,
    final_path=final_path,
)

## Run the optimisation

To run an optimisation, some parameters have to be set: the maximum number of generation for which the optimization algorithm will run ("max_ngen") and the size of the population ("offspring_size"). For the present example, the number of offspring as well as the maximum number of generations are kept voluntarily low, in practice, typical values would be for example offspring_size = 20 and max_ngen = 500.

For the optimizing algorithm there are three choices: "IBEA", "SO-CMA" and "MO-CMA". "MO-CMA" is recommended by default but the best algorithm can change depending on the problem.

In [ ]:
optimizer = "MO-CMA"
max_ngen = 5
opt_params = {
    'offspring_size': 3,
    'weight_hv': 0.4,
    'seed': 42
}

During the optimization, a checkpoint file, containing all the information about the optimisation process will be saved. It's path must also be provided:

In [ ]:
chkp_path = f"./checkpoints/checkpoint_{opt_params['seed']}.pkl"

Finally, the optimisation can be ran (warning: slow):

In [ ]:
_ = pipeline.optimize(
        max_ngen=max_ngen,
        stochasticity=False,
        copy_mechanisms=False,
        compile_mechanisms=True,
        opt_params=opt_params,
        optimizer=optimizer,
        checkpoint_path=chkp_path
    )

## Save the best models:

This next step will look at the checkpoint file generated during the optimisation and will save the best model in the final.json whose path was indicated above. This is necessary as for the "singlecell" access point, all subsequent operations (plotting, model management, etc) will except to read the e-models from this file.

In [ ]:
pipeline.store_best_model(
    stochasticity=False,
    copy_mechanisms=False,
    opt_params=opt_params,
    optimizer=optimizer,
    checkpoint_path=chkp_path
)

## Plot the convergence of the optimisation, the traces and the scores:

To look at the results of our optimisation, the following function can be used. All figures will be saved in the "figures" directory.

This first plotting function plots the minimum and average fitness as a function of the number of evaluation. It is useful to check if an optimisation succeeded. A succesfull optimisation should present a decreasing fitness.

In [ ]:
from bluepyemodel.emodel_pipeline import plotting

plotting.optimization(
    chkp_path, 
    figures_dir="./figures"
)

This next function reads the best models from the final.json and compute their responses and scores for the protocols used for optimization:

In [ ]:
pipeline.plot_models(figures_dir="./figures")

Using this function, it is also possible to compute the response to any protocols. 

Some eCodes are already defined and can be used as is (e.g: 'APWaveform', 'DeHyperpol', 'FirePattern', 'HyperDepol', 'IDrest', 'IV', 'PosCheops', Ramp', 'sAHP', 'SineSpec', 'SubWhiteNoise'):

In [ ]:
additional_protocols = {}
additional_protocols['PosCheops'] = {'type': 'StepThresholdProtocol'}

Protocols of custom length and amplitude can also be used. To know which parameters can be changed for the different protocol, have a look the classes defined in bluepyemodel/ecodes/.

For example, to define a custom step that doesnt not depend on the threshold current: 

(Note the change of the "type". The amplitude is specified in nA).

In [ ]:
additional_protocols['Step'] = {
    "type": "StepProtocol",
    'stimuli': {
        'amp': 0.4,
        'delay': 50,
        'duration': 20,
        'totduration': 100,
        'holding_current': -0.1
    } 
}

Another example: a ramp of custom length that depends of the threshold and holding currents.

The thresh_perc field is used to indicate an amplitude relative to the threshold current.

In [ ]:
additional_protocols['Ramp'] = {
    "type": "StepThresholdProtocol",
    'stimuli': {
        'thresh_perc': 0.8,
        'delay': 100,
        'duration': 500,
        'totduration': 800
    } 
}

Now let's plot the voltage responses to these stimuli (see the directory ./figures):

In [ ]:
pipeline.plot_models(
        figures_dir="./figures",
        additional_protocols=additional_protocols
    )